# Importando bibliotecas

In [1]:
import pandas as pd
import google.generativeai as genai
import os
import time
import re

# Análise inicial

In [2]:
conversas = pd.read_csv("conversas_leads.csv")
conversas.head()

,id,conversa
0,1,Assistente: Olá! Bem-vindo à Morada.ai. Como p...
1,2,Assistente: Olá! Seja bem-vindo à Morada.ai. C...
2,3,Assistente: Olá! Bem-vindo à Morada.ai. Como p...
3,4,Assistente: Olá! Bem-vindo à Morada.ai. Como p...
4,5,Assistente: Olá! Bem-vindo à Morada.ai. Como p...


In [3]:
conversas.shape

(10, 2)

In [4]:
empreendimentos = pd.read_csv("empreendimentos.csv")
empreendimentos.head()

,id,nome,descricao,localizacao,valor,quartos,banheiros,area,vagas,caracteristicas
0,1,Residencial Villa Verde,Condomínio fechado com apartamentos de alto pa...,"Jardim Europa, São Paulo - SP",850000,3,2,120,2,"Piscina, Academia, Salão de festas, Playground..."
1,2,Edifício Sky Tower,Apartamentos modernos com vista panorâmica da ...,"Centro, Rio de Janeiro - RJ",1200000,4,3,150,3,"Vista panorâmica, Smart home, Varanda gourmet,..."
2,3,Condomínio Solar das Palmeiras,Casas em condomínio fechado com amplos jardins...,"Alphaville, Barueri - SP",1800000,4,4,250,4,"Piscina privativa, Espaço gourmet, Jardim, Seg..."
3,4,Residencial Brisa Mar,Apartamentos à beira-mar com vista privilegiad...,"Beira Mar, Fortaleza - CE",950000,3,2,110,2,"Vista para o mar, Varanda gourmet, Piscina, Sa..."
4,5,Edifício Lumina,Apartamentos compactos e funcionais com design...,"Pinheiros, São Paulo - SP",550000,2,1,65,1,"Coworking, Lavanderia compartilhada, Bicicletá..."


In [5]:
empreendimentos.shape

(20, 10)

In [6]:
empreendimentos.columns

Index(['id', 'nome', 'descricao', 'localizacao', 'valor', 'quartos',
       'banheiros', 'area', 'vagas', 'caracteristicas'],
      dtype='object')

Após ter uma visão geral dos dois conjuntos de dados, e alguns testes, decidi seguir a implementação da primeira opção de projeto.

# Projeto

Para o projeto vou implementar a opção 1:

Extrator de Informações de Conversas: Desenvolva uma IA que extraia dados estruturados de conversas entre um assistente e um lead imobiliário.

Para isso nosso código terá 3 partes centrais:

 1- Extrair as informações do lead

 2- Parser em regex para salvar em um dicionário
 
 3- Transformar o dicionário em um .csv

## Configurando Gemini

In [7]:
# Configurando a chave da api
genai.configure(api_key=os.getenv('GEMINI_API_KEY'))

# Escolhendo o modelo
model = genai.GenerativeModel('gemini-2.0-flash')

## Definindo Funções

In [ ]:
#Parte 1: Extração de informação do lead
def extract_lead_info (model, lead):
    prompt = f'Extraia da conversa a seguir as seguintes informações em uma lista: nome do cliente, e-mail de contato, telefone de contato, localização desejada do imóvel, orçamento(número decimal, use o formato em ingles, sem separador de milhar), número de quartos, número de banheiros, tamanho desejado, número de vagas na garagem, e outras características necessárias (retorne com esse nome) para o imóvel\n\n segue a conversa\n\n {lead}'
    response = model.generate_content(prompt)

    return response.text


#Parte 2: Parser do retorno do modelo, para um dicionário
def get_lead_needs (leads, model):
    leads_dict = {}
    lead_count = 0
    for row in leads.iterrows():
        lead_count+=1 #Contagem do id do lead

        need = extract_lead_info(model, row[1]['conversa'])

        regex = r"\*\s*\*\*Nome do cliente:\*\*\s*([^\n*]+)\s*\*\s*\*\*E-mail de contato:\*\*\s*([^\n*]+)\s*\*\s*\*\*Telefone de contato:\*\*\s*([^\n*]+)\s*\*\s*\*\*Localização desejada do imóvel:\*\*\s*([^\n*]+)\s*\*\s*\*\*Orçamento:\*\*\s*([^\n*]+)\s*\*\s*\*\*Número de quartos:\*\*\s*([^\n*]+)\s*\*\s*\*\*Número de banheiros:\*\*\s*([^\n*]+)\s*\*\s*\*\*Tamanho desejado:\*\*\s*([^\n*]+)\s*\*\s*\*\*Número de vagas na garagem:\*\*\s*([^\n*]+)\s*\*\s*\*\*Outras características necessárias:\*\*\s*([^\n*]+)"
        #O regex acima foi escrito com auxílio da página https://regex101.com/ para casar com o padrão retornado pelo modelo
        matches = re.search(regex, need, flags=re.IGNORECASE)
        leads_dict[lead_count] = {'Nome cliente' : matches.group(1).strip(),
                                  'E-mail' : matches.group(2).strip(),
                                  'Telefone' : matches.group(3).strip(),
                                  'Local desejado' : matches.group(4).strip(),
                                  'Orcamento' : matches.group(5).strip(),
                                  'Quartos' : matches.group(6).strip(),
                                  'Banheiros' : matches.group(7).strip(),
                                  'Area' : matches.group(8).strip(),
                                  'Vagas' : matches.group(9).strip(),
                                  'Demais caracteristicas' : matches.group(10).strip()}
        time.sleep(5) #Tempo de espera para não ser bloqueado pela API
    return leads_dict

In [9]:
leads_dict = get_lead_needs(conversas, model)

In [ ]:
#Parte 3: Transformação do dicionário em um arquivo .csv
df_leads_info = pd.DataFrame.from_dict(leads_dict, orient = 'index')
df_leads_info.head()

,Nome cliente,E-mail,Telefone,Local desejado,Orcamento,Quartos,Banheiros,Area,Vagas,Demais caracteristicas
1,Carlos Eduardo Silva,carlos.silva@email.com,(11) 98765-4321,Pinheiros ou Vila Madalena (São Paulo),800000.00,2,Não especificado,Não especificado,Pelo menos 1,"Próximo ao metrô, espaço para escritório (devi..."
2,Ana Paula Mendes,ana.mendes@email.com,(11) 97654-3210,Alphaville ou Granja Viana,2000000.00,4,(Não especificado na conversa),(Não especificado na conversa),(Não especificado na conversa),"Piscina, proximidade a boas escolas."
3,Roberto Almeida,roberto.almeida@email.com,(48) 98877-6655,"Florianópolis, região central ou próximo às pr...",500000.00,1 ou studio,Não especificado,Não especificado,Não especificado,"Imóvel para investimento, preferência para alu..."
4,Marcelo Gomes,marcelo.gomes@email.com,(31) 99988-7766,"Belo Horizonte, Savassi",3500.00,3,Não especificado,Não especificado,Pelo menos 1,"Academia no prédio, proximidade à PUC, disponi..."
5,Fernando Costa,fernando.costa@email.com,(11) 96655-4433,Atibaia ou Bragança Paulista,400000.00,Não especificado,Não especificado,Pelo menos 1000 m²,Não especificado,"Em condomínio fechado, com vista bonita."


In [11]:
df_leads_info.to_csv("Leads_info.csv")